In [24]:
from openai import OpenAI
import os
import re
import base64
import requests
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from IPython.display import FileLink, display, HTML
from dotenv import load_dotenv
from random import shuffle, randint, choice, random
from collections import Counter
# Load API key
_ = load_dotenv("../comm4190_F25/01_Introduction_and_setup/.env")
client = OpenAI()

In [3]:
# prompt for brainstorming ideas
SYSTEM_PROMPT = """
You are a product design expert in a robotics company identifying new opportunities in the market for innovative consumer gadgets.

Please return a list of 10 such items in the following format:
1. Item Name
- Description
- Key Selling Points
"""

In [4]:
chat_window = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT
    }
]

response = client.chat.completions.create(
    model = "gpt-4o",
    messages = chat_window,
    store = False
)
product_ideas = response.choices[0].message.content
filename = 'product_ideas.txt'
with open(filename, 'w') as file:
    file.write(product_ideas)

FileLink(filename)

/Commjhub/jupyterhub/home/ezou626/comm4190_F25_Product_Task/product_ideas.txt

In [8]:
# list products for retrieval later

pattern = re.compile(
    r"\d+\.\s+(.*?)\n- Description:\s*(.*?)\n- Key Selling Points:\s*(.*?)(?:\n|$)",
    re.DOTALL
)

def parse_product_ideas(product_list: str):
    return pattern.findall(product_list) #re.findall(r'\d+\.\s+(.*)', product_list)

parsed_product_ideas = parse_product_ideas(product_ideas)
# print(parsed_product_ideas)

In [6]:
selector_prompt_creator = lambda x: f"""
You are panelist {x} on a group of internal product evaluators that will help determine the direction a robotics company takes in investing in a new consumer gadget.
Your objective is to develop a ranking for the list of products provided to you based on your unique experience.
Please return the product numbers in a comma separated list, with most preferred product first.
"""

def get_voting_results(panelist_id):
    chat_window = [
        {
            "role": "system",
            "content": selector_prompt_creator(panelist_id)
        },
        {
            "role": "user",
            "content": "Here are the product ideas:\n" + product_ideas
        }
    ]
    
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = chat_window,
        store = False
    )
    
    return response.choices[0].message.content

results = process_map(get_voting_results, range(1, 11))

for result in results:
    print(result)

  0%|          | 0/10 [00:00<?, ?it/s]

1, 3, 10, 4, 5, 9, 2, 6, 8, 7
3, 1, 4, 5, 2, 9, 6, 10, 7, 8
3, 1, 9, 10, 4, 5, 2, 8, 6, 7
1, 4, 5, 9, 1, 10, 2, 3, 8, 6, 7
3, 10, 4, 1, 2, 9, 6, 5, 8, 7
1, 3, 10, 4, 9, 6, 2, 8, 1, 7
3, 1, 10, 9, 2, 4, 6, 5, 8, 7
3, 10, 4, 1, 9, 2, 5, 8, 6, 7
1, 4, 3, 5, 2, 10, 9, 6, 8, 7
3, 1, 10, 6, 4, 5, 9, 8, 2, 7


In [7]:
# parse results
votes = [[int(i) for i in result.split(", ")] for result in results]

In [30]:
# evaluate ranking
from collections import Counter

def ranked_choice_ranking(votes):
    votes = [list(ballot) for ballot in votes]  # copy to avoid modifying input
    candidates = set(c for ballot in votes for c in ballot)
    elimination_order = []

    while len(candidates) > 1:
        # Count first-choice votes
        first_choices = [ballot[0] for ballot in votes if ballot]
        counts = Counter(first_choices)

        # Find lowest-vote candidate(s)
        min_votes = min(counts[c] for c in candidates)
        eliminated = [c for c in candidates if counts.get(c, 0) == min_votes]

        # Record elimination
        elimination_order.extend(eliminated)

        # Remove eliminated from ballots
        for ballot in votes:
            for e in eliminated:
                if e in ballot:
                    ballot.remove(e)

        candidates -= set(eliminated)

    # Last remaining candidate is the winner
    elimination_order.extend(candidates)

    # Reverse gives 1st → last
    return elimination_order[::-1]

4. Sleep Optimization Robot
A bedside gadget that analyzes sleep patterns and dynamically adjusts room conditions (lighting, sound, temperature) for optimal rest.

5. Interactive Language Tutor
A portable AI robot that teaches languages through conversation and interactive storytelling.

3. Remote Pet Pal
An interactive pet companion that plays, feeds, and monitors pets remotely via smartphone app.

10. Augmented Reality Fashion Stylist
A wearable AR device that suggests clothing and accessory matches, visualizing them on the user through augmented reality.

9. Virtual Fitness Coach
A holographic fitness trainer that offers personalized workouts and real-time feedback in your living room.

8. Eco-Friendly Home Cleaner
An autonomous robot that uses natural, non-toxic methods for cleaning and disinfecting household surfaces.

7. AI Personal Safety Drone
A pocket-sized drone that accompanies the user to provide real-time monitoring and emergency alerts in potentially dangerous situations.

In [ ]:
ranking = ranked_choice_ranking(votes)
for i in ranking:
    name, description, _ = parsed_product_ideas[i - 1]
    print(str(i) + '.', name)
    print(description + '\n')

In [32]:
advancing_products = 4
products = [parsed_product_ideas[i - 1] for i in ranking[:advancing_products]]
for name, description, _ in products:
    print(name)
    print(description + '\n')

Sleep Optimization Robot
A bedside gadget that analyzes sleep patterns and dynamically adjusts room conditions (lighting, sound, temperature) for optimal rest.

Interactive Language Tutor
A portable AI robot that teaches languages through conversation and interactive storytelling.

Remote Pet Pal
An interactive pet companion that plays, feeds, and monitors pets remotely via smartphone app.

Augmented Reality Fashion Stylist
A wearable AR device that suggests clothing and accessory matches, visualizing them on the user through augmented reality.



In [22]:
# run this cell to inject our own ideas

products = [
    [
        "Sleep Optimization Robot",
        "A bedside gadget that analyzes sleep patterns and dynamically adjusts room conditions (lighting, sound, temperature) for optimal rest.",
        None
    ],
    [
        "Interactive Language Tutor",
        "A portable AI robot that teaches languages through conversation and interactive storytelling.",
        None
    ],
    [
        "Remote Pet Pal",
        "An interactive pet companion that plays, feeds, and monitors pets remotely via smartphone app.",
        None
    ],
    [
        "Augmented Reality Fashion Stylist",
        "A wearable AR device that suggests clothing and accessory matches, visualizing them on the user through augmented reality.",
        None
    ]
]

In [28]:
marketing_consulting_companies = ["EY", "McKinsey", "BCG", "Bain"]

def create_marketing_assets_with_image(data):
    product_name, product_description, company = data
    CONSULTANT_PROMPT = f"""
    You are a consultant from {company} advising a robotics company launching a new consumer gadget.
    Your objective is to develop a brand name and tag line for the new product.
    Please leverage your company's unique strengths and perspective to complete this task.
    You should format your response as "[brand name]|[tagline]"
    """

    chat_window = [
        {
            "role": "system",
            "content": CONSULTANT_PROMPT
        },
        {
            "role": "user",
            "content": f'The product is "{product_name}" and it is "{product_description}"'
        }
    ]

    # Get brand name and tagline from GPT
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=chat_window,
        store=False
    )
    brand_tagline = response.choices[0].message.content.strip()
    brand_name, tagline = [part.strip() for part in brand_tagline.split('|', 1)]

    # Prepare a prompt for DALL·E using the brand and tagline
    image_prompt = f"An artistic, futuristic product design for '{brand_name}', a gadget described as: {tagline}"

    # Generate image with DALL·E
    image_response = client.images.generate(
        model="dall-e-3",
        prompt=image_prompt,
        n=1,
        size="1024x1024"
    )

    image_url = image_response.data[0].url

    # Return all marketing assets including image URL
    return {
        "brand_name": brand_name,
        "tagline": tagline,
        "image_url": image_url
    }

# product_assets now contains brand name, tagline, and image URL per company per product

In [29]:
# THIS IS A VERY EXPENSIVE CELL TO RUN
from functools import partial

product_assets = []
for name, description, _ in tqdm(products):
    generate_for_product = partial(create_marketing_assets_with_image, name, description)
    results = process_map(generate_for_product, marketing_consulting_companies)
    product_assets.append(results)

100%|██████████| 4/4 [05:27<00:00, 81.90s/it]


In [30]:
for (name, description, _), assets in zip(products, product_assets):
    print(name)
    print(description)
    print()
    for index, assets in enumerate(assets):
        brandname = assets["brand_name"]
        tagline = assets["tagline"]
        image_url = assets["image_url"]
        print(f"{index + 1}.", brandname)
        print(tagline)
        display(HTML(f"<a href = {image_url}>Image Link</a>"))
        print()
    print("\n")

Sleep Optimization Robot
A bedside gadget that analyzes sleep patterns and dynamically adjusts room conditions (lighting, sound, temperature) for optimal rest.

1. DreamSync
"Rest Revolutionized: Personalized Comfort, Perfect Sleep"



2. SlumbraSync
"Align Your Nights, Awaken Brighter"



3. DreamAura
Optimize Your Nights, Transform Your Days



4. LunaHarmony
"Transform Your Night, Elevate Your Day."





Interactive Language Tutor
A portable AI robot that teaches languages through conversation and interactive storytelling.

1. LingoBot
"Unlock the World, One Story at a Time"


...


4. PetSync
"Your Pet, Your Control – Anytime, Anywhere"





Augmented Reality Fashion Stylist
A wearable AR device that suggests clothing and accessory matches, visualizing them on the user through augmented reality.

1. StyloSphere
"Unleash Your Style in Every Dimension"



2. StyleSync
Elevate Your Wardrobe, Effortlessly



3. StyleSync
Effortless Elegance, Augmented for You



4. FashionFusion
Elevate Your Style, Instantly
